# Cohere

This notebook covers how to get started with [Cohere chat models](https://cohere.com/chat).

Head to the [API reference](https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.cohere.ChatCohere.html) for detailed documentation of all attributes and methods.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `cohere` package itself. We can install these with:

```bash
pip install -U langchain-community cohere
```

We'll also need to get a [Cohere API key](https://cohere.com/) and set the `COHERE_API_KEY` environment variable:

In [1]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

ChatCohere supports all [ChatModel](/docs/modules/model_io/chat/) functionality:

In [2]:
from langchain_community.chat_models import ChatCohere
from langchain_core.messages import HumanMessage

In [8]:
chat = ChatCohere()

In [ ]:
messages = [HumanMessage(content="knock knock")]
chat.invoke(messages)

In [ ]:
await chat.ainvoke(messages)

In [ ]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

In [ ]:
chat.batch([messages])

## Chaining

You can also easily combine with a prompt template for easy structuring of user input. We can do this using [LCEL](/docs/expression_language)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat

In [ ]:
chain.invoke({"topic": "bears"})

## Search Queries

To generate search queries for retrievers use search_queries_only

In [11]:
messages = [HumanMessage(content="What is the highest point and lowest point on earth?")]
res = chat.invoke(messages, search_queries_only=True)
print(res.additional_kwargs["search_queries"])

[{'text': 'highest and lowest point on earth?', 'generation_id': 'dc1ca1ae-0724-4af3-9570-ee42e063c177'}]


## Perform RAG

To perform RAG generate search queries, fetch the documents using a retriever 

In [17]:
from langchain.retrievers import CohereRagRetriever
from langchain.retrievers import WikipediaRetriever
from langchain_community.chat_models import ChatCohere

# User query we will use for the generation
user_query = "What is the highest point and lowest point on earth?"
messages = [HumanMessage(user_query)]
# Load the cohere chat model
cohere_chat_model = ChatCohere()
# Create the cohere rag retriever using the chat model
rag = CohereRagRetriever(llm=cohere_chat_model, connectors=[])
# Get search queries 
res = chat.invoke(messages, search_queries_only=True)
# Create the wikipedia retriever
wiki_retriever = WikipediaRetriever()
# Get the relevant documents from wikipedia
wiki_docs = []
for query in res.additional_kwargs["search_queries"]:
    wiki_docs.extend(wiki_retriever.get_relevant_documents(query["text"]))
# Get the cohere generation from the cohere rag retriever
docs = rag.get_relevant_documents(user_query ,source_documents=wiki_docs)
# Print the final generation 
answer = docs[-1].page_content
print(answer)
# Print the final citations 
citations = docs[-1].metadata['citations']
print(citations)
# Print the documents
for doc in docs[:-1]:
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")

The highest point on Earth is Mount Everest, which sits at 8,848 meters (29,029 feet) above sea level. The lowest point is the Mariana Trench, which is 10,920 meters (35,798 feet) below sea level. 

For countries, Lesotho has the highest low point at 1,400 meters (4,593 feet). Countries with very low high points include Maldives at 5 meters (16 feet), Tuvalu, and the Marshall Islands at 10 meters (33 feet).
[{'start': 30, 'end': 43, 'text': 'Mount Everest', 'document_ids': ['doc-1']}, {'start': 59, 'end': 85, 'text': '8,848 meters (29,029 feet)', 'document_ids': ['doc-1']}, {'start': 127, 'end': 141, 'text': 'Mariana Trench', 'document_ids': ['doc-1']}, {'start': 152, 'end': 179, 'text': '10,920 meters (35,798 feet)', 'document_ids': ['doc-1']}, {'start': 214, 'end': 221, 'text': 'Lesotho', 'document_ids': ['doc-0']}, {'start': 230, 'end': 276, 'text': 'highest low point at 1,400 meters (4,593 feet)', 'document_ids': ['doc-0']}, {'start': 322, 'end': 409, 'text': 'Maldives at 5 meters 